## Problema e solução

> O início da aplicação da ACO envolve a conversão do problema escolhido em um grafo, no qual a rota, indicando a trilha mais intensa de feromônios, corresponde à solução desejada. O objetivo é determinar um percurso otimizado através desse grafo.

> Nesse contexto, a escolha natural é o clássico Problema do Caixeiro Viajante. Este problema consiste em encontrar a rota mais curta para visitar um conjunto de cidades, passando por cada uma exatamente uma vez e retornando à cidade de origem.

> O espaço de estados para esse problema pode ser visualizado como um grafo completamente conexo, em que os vértices representam as cidades e as arestas denotam as vias entre elas, cada uma associada a uma distância (ou custo).

> Um código fornecido abaixo é uma visão menos abstraida do Problema do Caixeiro Viajante, com cada etapa da solução.

In [2]:
# Importando pacotes e módulos
import numpy as np
from numpy import inf

In [3]:
# Geração do grafo para o problema do caixeiro (nesse caso, manualmente)
d = np.array([[0,10,12,11,14],
              [10,0,13,15,8],
              [12,13,0,9,14],
              [11,15,9,0,16],
              [14,8,14,16,0]])
                

# Configurando variáveis
 # (Iterações, Número de formigas, número de cidades, evaporação )
iteration = 100
n_ants = 5  #m
n_citys = 5 #n
evaporation = 0.5
alpha = 1
beta = 2

In [4]:
# Calculando a visibilidade da próxima visibilidade da cidade(i,j)=1/d(i,j)
visibility = 1/d
visibility[visibility == inf] = 0

C:\Users\pc\AppData\Local\Temp\ipykernel_9516\2113696052.py:2: RuntimeWarning: divide by zero encountered in divide
  visibility = 1/d


In [5]:
# Inicializando feromonios no caminho das cidades
pheromne = .1*np.ones((n_ants,n_citys))

# Inicializando a rota das formigas com tamanho rute(n_ants,n_citys+1)
rute = np.ones((n_ants,n_citys+1))

In [6]:
# Procesando solução
for ite in range(iteration):

    rute[:,0] = 1

    for i in range(n_ants):

        temp_visibility = np.array(visibility)

        for j in range(n_citys-1):
            #print(rute)

            combine_feature = np.zeros(5)
            cum_prob = np.zeros(5)

            cur_loc = int(rute[i,j]-1)

            temp_visibility[:,cur_loc] = 0

            p_feature = np.power(pheromne[cur_loc,:],beta)
            v_feature = np.power(temp_visibility[cur_loc,:],alpha)

            p_feature = p_feature[:,np.newaxis]
            v_feature = v_feature[:,np.newaxis]

            combine_feature = np.multiply(p_feature,v_feature)

            total = np.sum(combine_feature)

            probs = combine_feature/total

            cum_prob = np.cumsum(probs)
            #print(cum_prob)
            r = np.random.random_sample()
            #print(r)
            city = np.nonzero(cum_prob>r)[0][0]+1
            #print(city)

            rute[i,j+1] = city

        left = list(set([i for i in range(1,n_citys+1)])-set(rute[i,:-2]))[0]

        rute[i,-2] = left

    rute_opt = np.array(rute)

    dist_cost = np.zeros((n_ants,1))

    for i in range(n_ants):

        s = 0
        for j in range(n_citys-1):

            s = s + d[int(rute_opt[i,j])-1,int(rute_opt[i,j+1])-1]

        dist_cost[i]=s

    dist_min_loc = np.argmin(dist_cost)
    dist_min_cost = dist_cost[dist_min_loc]

    best_route = rute[dist_min_loc,:]
    pheromne = (1-evaporation)*pheromne

    for i in range(n_ants):
        for j in range(n_citys-1):
            dt = 1/dist_cost[i]
            pheromne[int(rute_opt[i,j])-1,int(rute_opt[i,j+1])-1] = pheromne[int(rute_opt[i,j])-1,int(rute_opt[i,j+1])-1] + dt

C:\Users\pc\AppData\Local\Temp\ipykernel_9516\1922287799.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pheromne[int(rute_opt[i,j])-1,int(rute_opt[i,j+1])-1] = pheromne[int(rute_opt[i,j])-1,int(rute_opt[i,j+1])-1] + dt


In [27]:
# Exibindo resultado
print('route of all the ants at the end :')
print(rute_opt)
print()
print('best path :',best_route)
print('cost of the best path',int(dist_min_cost[0]) + d[int(best_route[-2])-1,0])

route of all the ants at the end :
[[1. 3. 4. 5. 2. 1.]
 [1. 3. 4. 5. 2. 1.]
 [1. 3. 4. 5. 2. 1.]
 [1. 3. 4. 5. 2. 1.]
 [1. 3. 4. 5. 2. 1.]]

best path : [1. 3. 4. 5. 2. 1.]
cost of the best path 55
